Le but de se code est de scrapper les prix littéraires en France entre 2019 et 2023

In [9]:
# on importe la bibliothèque requests
import requests
from bs4 import BeautifulSoup # on importe BeautifulSoup
import pandas as pd

On crée un dataframe vide avec comme colonnes les infos qu'on va importer

In [10]:
colonnes = ['Année', 'Prix', 'Auteur', 'Titre']

# Créer un DataFrame vide avec ces colonnes
df = pd.DataFrame(columns=colonnes)

print(df)

Empty DataFrame
Columns: [Année, Prix, Auteur, Titre]
Index: []


On scrape

In [11]:
def scrape_wikipedia_page_france(year):
    global df
    url = f"https://fr.wikipedia.org/wiki/Prix_litt%C3%A9raires_{year}"
    response = requests.get(url)
    response.encoding = "utf-8"  # On garantit le bon encodage

    if response.status_code == 200:  # On vérifie si la requête est un succès
        html_content = response.text
        soup = BeautifulSoup(html_content, 'html.parser')

        # On recherche la section France
        france_section = soup.find('h2', string="France")
        if france_section:
            # La liste <ul> qui suit contient les prix pour la France
            france_list = france_section.find_next('ul')

            if france_list:
                # On recherche tous les <li> dans la liste
                prizes = france_list.find_all('li')

                # On extrait les données
                for prize in prizes:
                    # Nom du prix
                    prize_name = prize.find('a')
                    prize_name = prize_name.text.strip() if prize_name else None

                    # Auteur
                    author = prize.find_all('a')
                    author = author[1].text.strip() if len(author) > 1 else None

                    # Titre
                    title = prize.find('i')
                    title = title.text.strip() if title else None

                    # On ajoute les résultats si les données sont complètes
                    if prize_name and author and title:
                        new_row = {'Année': year, 'Prix': prize_name, 'Auteur': author, 'Titre': title}
                        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
        else:
            print(f"Section 'France' non trouvée pour l'année {year}")
    else:
        print(f"Erreur lors du scraping de l'année {year}: Status code {response.status_code}")

    return df

In [12]:
for year in range(2019, 2024):
    scrape_wikipedia_page_france(year)

In [13]:
df.head(5)

,Année,Prix,Auteur,Titre
0,2019,Prix Femina,Par les routes,Par les routes
1,2019,Prix Femina étranger,Ordesa,Ordesa
2,2019,Prix Femina essai,Emmanuelle Lambert,Giono furioso
3,2019,Prix Femina des lycéens,La Chaleur,La Chaleur
4,2019,Prix Goncourt,Tous les hommes n'habitent pas le monde de la ...,Tous les hommes n'habitent pas le monde de la ...


Création d'un fichier CSV afin de vérifier s'il n'y a pas d'incohérences

In [35]:
df.to_csv("prix_litteraires.csv", index=False)

PARTIE: Corriger les incohérences de la table

Création de fonctions

In [ ]:
import requests
import re
from bs4 import BeautifulSoup as soup

# Fonction pour rechercher un livre sur Babelio et accéder à la page du premier résultat
def search_wikipedia(book_title, year):
    #reformulation du titre en différente partie
    parts = re.findall(r'\w+|[^\w\s]', book_title)

    # URL de recherche sur Livraddict
    search_title = '%20'.join(parts)
    lower_search_title = '%20'.join(parts).lower()
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    search_url = f'https://www.bing.com/search?q=Livre%20{search_title}%20{year}&qs=n&form=QBRE&sp=-1&ghc=1&lq=0&pq=livre%20{lower_search_title}%20{year}&sc=11-30&sk=&cvid=88409FB04E76408A93DE43207AD5027F&ghsh=0&ghacc=0&ghpl='

    # Faire une requête pour récupérer la page des résultats de recherche
    response = requests.get(search_url, headers=headers).content
    response_text = response.decode('utf-8')
    # Trouver le premier élément correspondant au lien d'un résultat
    page = soup(response_text, "html.parser")
    first_result = page.find('li', class_='b_algo')  # Trouve le premier résultat
    if first_result:
        link = first_result.find('a', href=True)  # Trouve le lien à l'intérieur de <a>
        if link:
            href = link['href']
            # Si c'est une URL relative, complète-la
            if href.startswith('/wiki'):
                href = 'https://fr.wikipedia.org' + href
            return href
        else:
                print("Pas de lien trouvé dans le premier résultat.")
                return None
    else:
            print("Pas de résultats trouvés.")
            return None

RECHERCHE SUR GALLIMARD PCQ TROP DE BLOCAGES SUR LES MOTEURS DE RECHERCHE

In [32]:
import time
import re
import requests
from bs4 import BeautifulSoup as soup

def search_gallimard(book_title, year):
    #reformulation du titre en différente partie
    parts = re.findall(r'\w+|[^\w\s]', book_title)
    book_title_gen = book_title.lower().strip()

    # URL de recherche sur Livraddict
    search_title = '+'.join(parts)
    lower_search_title = '%20'.join(parts).lower()
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:118.0) Gecko/20100101 Firefox/118.0'}
    search_url = f'https://www.gallimard.fr/catalogue?search_api_fulltext={search_title}&field_date_de_parution_1%5Bdate%5D={year}-01-01&field_date_de_parution_2%5Bdate%5D={year}-12-31&field_prix%5Bmin%5D=&field_prix%5Bmax%5D=&title_3=&items_per_page=20&sort_bef_combine=field_identifiant_catalogue_ASC'
    time.sleep(5)

    # Faire une requête pour récupérer la page des résultats de recherche
    response = requests.get(search_url, headers=headers).content
    response_text = response.decode('utf-8')
    # Trouver le premier élément correspondant au lien d'un résultat
    page = soup(response_text, "html.parser")
    results = page.find_all("div", class_ = "card--ouvrage")
    author = ''
    for i in range(len(results)):
        result = results[i]
        div_title = result.find("div", class_ = "title")
        title = div_title.text
        title_gen = title.lower().strip()
        if book_title_gen == title_gen:
            div_author = result.find('div', class_='paragraph paragraph--type--auteur paragraph--view-mode--links')
            a_author = div_author.find("a")
            author = a_author.text
    if author == '':
        author = "Pas trouvé sur Gallimard"
    return author

ESSAI

In [28]:
search_gallimard('Par les routes', '2019')

'Sylvain Prudhomme'

Fonction pour récupérer l'information sur l'auteur depuis la page wikipedia je pense que je vais enlever tout ça

In [25]:
def recup_info(url):
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(url, headers=headers).text
    if response == None:
        author = "problème d'URL"
    page = BeautifulSoup(response, "html.parser")
    table = page.find('table', {'class': 'infobox'})
    if table == None:
        author = "Erreur de trouvaille de la table wikipédia" 
    else: 
        rows = table.find_all('tr')
        if rows == None:
            author = "Pas de catégories trouvées dans la table"
        else:
            tr_author = rows.find("tr")
            if tr_author == None:
                author = "Pas de rubrique auteur dans la table"
            else:
                td_author = tr_author.find("td")
                if td_author == None:
                    author = "Pas de nom dans la ligne auteur (td)"
                else:
                    a_autor = td_author.find("a")
                    if a_autor == None :
                        author = "Pas de nom dans la ligne auteur (a)"
                    else:
                        author = a_autor.get("title")
                        if author == None:
                            author = "Pas de nom dans la ligne auteur (title)"
        
    return author



In [66]:
import time
import re
import requests
from bs4 import BeautifulSoup as soup

def search_maison_du_livre(book_title, year):
    #reformulation du titre en différente partie
    parts = re.findall(r'\w+|[^\w\s]', book_title)
    book_title_gen = book_title.lower().strip()

    # URL de recherche sur Livraddict
    search_title = '+'.join(parts)
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:118.0) Gecko/20100101 Firefox/118.0'}
    search_url = f'https://www.maisondulivre.com/listeliv.php?base=paper&mots_recherche={search_title}+{year}'
    time.sleep(5)

    # Faire une requête pour récupérer la page des résultats de recherche
    response = requests.get(search_url, headers=headers).content
    response_text = response.decode('utf-8')

    # Trouver le premier élément correspondant au lien d'un résultat
    page = soup(response_text, "html.parser")
    liste_livres = page.find('ul', {'id': 'liste_livres'})
    livres = liste_livres.find_all("li")
    author = ''
    for i in range(len(livres)):
        livre = livres[i]
        h2_titre = livre.find("h2", class_="livre_titre")
        if h2_titre:
            a_titre = h2_titre.find("a")
            titre = a_titre.get_text(strip=True)
            title_adapted = re.sub(r'\s?\(.*?\)', '', titre)
            title_gen = title_adapted.lower().strip()
            if title_gen == book_title_gen:
                h2_auteur = livre.find("h2", class_ = "livre_auteur")
                if h2_auteur == None:
                    auteur = "Auteur non renseigné"
                else:
                    a_auteur = h2_auteur.find("a")
                    author = a_auteur.get_text(strip=True)
    if author == '':
        author = "Livre pas trouvé"
    return author

In [63]:
search_maison_du_livre('Ordesa', '2019')

'Manuel Vilas'

In [31]:
def search_author(row):
    if row["Auteur"] == row["Titre"]:
        author = search_gallimard(row["Titre"], row["Année"])
        if author == None:
            author = "Problème dans la fonction?"
        return author
    else:
        return row["Auteur"]

Fonction globale allant chercher l'auteur de l'ouvrage quand ce dernier n'est pas indiqué. Attention, l'opération prend à peu près 8 minutes

In [33]:
df['Auteur'] = df.apply(search_author, axis=1)

In [34]:
df.head(5)

,Année,Prix,Auteur,Titre
0,2019,Prix Femina,Sylvain Prudhomme,Par les routes
1,2019,Prix Femina étranger,Pas trouvé sur Gallimard,Ordesa
2,2019,Prix Femina essai,Emmanuelle Lambert,Giono furioso
3,2019,Prix Femina des lycéens,Pas trouvé sur Gallimard,La Chaleur
4,2019,Prix Goncourt,Pas trouvé sur Gallimard,Tous les hommes n'habitent pas le monde de la ...


Pour compléter ce qu'on a déjà fait sur Gallimard, on va utiliser un scraping sur le site de la maison du livre

In [67]:
def complete_author(row):
    if row["Auteur"] == "Pas trouvé sur Gallimard":
        author = search_maison_du_livre(row["Titre"], row["Année"])
        if author == None:
            author = "Problème dans la fonction?"
        return author
    else:
        return row["Auteur"]

In [68]:
df['Auteur'] = df.apply(complete_author, axis=1)

In [69]:
df.head(10)

,Année,Prix,Auteur,Titre
0,2019,Prix Femina,Sylvain Prudhomme,Par les routes
1,2019,Prix Femina étranger,Manuel Vilas,Ordesa
2,2019,Prix Femina essai,Emmanuelle Lambert,Giono furioso
3,2019,Prix Femina des lycéens,Victor Jestin,La Chaleur
4,2019,Prix Goncourt,Jean-Paul Dubois,Tous les hommes n'habitent pas le monde de la ...
5,2019,Prix Goncourt du premier roman,Marie Gauthier,Court vêtue
6,2019,Prix Goncourt des lycéens,Karine Tuil,Les Choses humaines
7,2019,Prix Goncourt de la nouvelle,Caroline Lamarche,Nous sommes à la lisière
8,2019,Prix Goncourt de la biographie,Emily Dickinson,"Manifeste incertain, volume 7 : Emily Dickinso..."
9,2019,Choix Goncourt de la Pologne,Louis-philippe Dalembert,Mur Méditerranée


Regardons combien de valeurs sont pas encore expliquées

In [70]:
df.to_csv("prix_litteraires.csv", index=False)

Je remplace à la main pour les dernières valeurs

In [ ]:
df.loc[df['Titre'] == "J'ai oublié", 'Auteur'] = 'Bulle Ogier'
df.loc[df['Titre'] == "Une vieille histoire. Nouvelle version", 'Auteur'] = 'Jonathan Littell'
df.loc[df['Titre'] == "Les Grands Cerfs", 'Auteur'] = 'Claudie Hunzinger'
df.loc[df['Titre'] == "Anatèm", 'Auteur'] = 'Neal Stephenson'
df.loc[df['Titre'] == "Le Cœur synthétique", 'Auteur'] = 'Chloé Delaume'
df.loc[df['Titre'] == "Terra Ignota", 'Auteur'] = 'Ada Palmer'
df.loc[df['Titre'] == "Histoire du fils", 'Auteur'] = 'Marie-Hélène Lafon'
df.loc[df['Titre'] == "Adios Cow Boy", 'Auteur'] = 'Olja Savičević Ivančević'
df.loc[df['Titre'] == "La Grâce", 'Auteur'] = 'Thibault de Montaigu'
df.loc[df['Titre'] == "Le Voyant d'Étampes", 'Auteur'] = 'Abel Quentin'
df.loc[df['Titre'] == "Héritage", 'Auteur'] = 'Miguel Bonnefoy'
df.loc[(df['Titre'] == 'Femme Actuelle') & (df['Année'] == 2019), ['Titre', 'Auteur', 'Prix']] = ['Au bout de la nuit', 'Bruno Bouzounie', "Grand prix Roman de l'été de Femme Actuelle"]
df.loc[(df['Titre'] == 'Femme Actuelle') & (df['Année'] == 2020), ['Titre', 'Auteur', 'Prix']] = ['La Conjonction dorée', 'Benoît Sagaro', "Grand prix Roman de l'été de Femme Actuelle"]
df.loc[(df['Titre'] == 'Femme Actuelle') & (df['Année'] == 2021), ['Titre', 'Auteur', 'Prix']] = ['Pour exister encore', 'Martine Duchesne', "Grand prix Roman de l'été de Femme Actuelle"]
df.loc[(df['Titre'] == 'Femme Actuelle') & (df['Année'] == 2022), ['Titre', 'Auteur', 'Prix']] = ["Le journal de Betty Swan", 'Cindy Valt', "Grand prix Roman de l'été de Femme Actuelle"]
df.loc[(df['Titre'] == 'Femme Actuelle') & (df['Année'] == 2023), ['Titre', 'Auteur', 'Prix']] = ['Hauteclaire', 'Dominique Delplace', "Grand prix Roman de l'été de Femme Actuelle"]

df.loc[(df['Prix'] == 'Prix littéraire du Monde') & (df['Année'] == 2020), ['Titre', 'Auteur']] = ['Elle a menti pour les ailes', 'Francesca Serra']
df.loc[(df['Prix'] == 'Prix Eugène-Dabit du roman populiste') & (df['Année'] == 2021), 'Auteur'] =  'Samira Sedira'
df.loc[(df['Titre'] == 'Mon territoire') & (df['Année'] == 2020), ['Prix', 'Auteur']] = ['Grand prix des lectrices (Elle)', 'Tess Sharpe']
df.loc[(df['Titre'] == 'Les Paupières des poissons') & (df['Année'] == 2019), ['Prix', 'Auteur']] = ['Prix Maya du livre animaliste', 'Fanny Vaucher, Sébastien Moro']
df.loc[(df['Titre'] == 'Traverser l’invisible') & (df['Année'] == 2019), ['Prix', 'Auteur']] = ['Prix Maya du livre animaliste', 'Fanny Vaucher, Sébastien Moro']


On vérifie qu'il n'y a bien plus aucune anomalie

In [84]:
erreur_recherche = ["Livre pas trouvé"]
df_verif =  df[df["Auteur"].isin(erreur_recherche)]
print(df_verif)

Empty DataFrame
Columns: [Année, Prix, Auteur, Titre]
Index: []


Je vérifie sur le csv qu'il n'y a plus d'anomalie

In [87]:
df.to_csv("prix_litteraires.csv", index=False)

In [17]:
recup_info('https://fr.wikipedia.org/wiki/La_Mer_des_monstres')

[<tr>
 <td class="entete universite italique" colspan="2" style="background-color:#7DA7D9;color:black;">La Mer des monstres<style data-mw-deduplicate="TemplateStyles:r160407116">.mw-parser-output .entete.universite{background-image:url("//upload.wikimedia.org/wikipedia/commons/4/42/Picto_infobox_book.png")}</style>
 </td></tr>,
 <tr><td colspan="2" style="display:none;">
 </td></tr>,
 <tr>
 <th scope="row">Auteur
 </th>
 <td><a href="/wiki/Rick_Riordan" title="Rick Riordan">Rick Riordan</a>
 </td>
 </tr>,
 <tr>
 <th scope="row">Pays
 </th>
 <td><span class="nowrap"><span class="datasortkey" data-sort-value="États unis"><span class="flagicon"><span class="mw-image-border noviewer" typeof="mw:File"><a class="mw-file-description" href="/wiki/Fichier:Flag_of_the_United_States.svg" title="Drapeau des États-Unis"><img alt="Drapeau des États-Unis" class="mw-file-element" data-file-height="650" data-file-width="1235" decoding="async" height="11" src="//upload.wikimedia.org/wikipedia/commons/th

In [16]:
def recup_info(url):
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(url, headers=headers).content
    page = BeautifulSoup(response, "html.parser")
    table = page.find('table', {'class': 'infobox'})
    rows = table.find_all('tr')
    return rows